In [1]:
# Install Python libraries
%pip install yt_dlp
%pip install audio-separator onnxruntime
%pip install selenium
%pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
def create_folder(base_path, artist, title):
   # Get the artist and title of the song and delete special characters
  artist = artist.replace(" ", "_").replace("/", "_")
  title = title.replace(" ", "_").replace("/", "_")

  # Create folder for the song
  folder = f"{artist}/{title}"
  base_path = os.path.join(base_path, folder)
  if not os.path.exists(base_path):
    os.makedirs(base_path)

  return base_path

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = ChromeService(ChromeDriverManager().install())

def get_url_music(query_song):
  driver = webdriver.Chrome(service=service, options=options)
  try:
    # query_song = "NSYNC Bye Bye Bye"
    url = "https://music.youtube.com/search?q=" + query_song
    # Open a webpage
    driver.get(url)

    html_source = driver.page_source

    # Locate the element containing the dynamic href attribute
    element = driver.find_element(By.XPATH, '//a[contains(@href, "watch?v=")]')

    # Extract the href attribute value
    href_value = element.get_attribute('href')

    print(href_value)
    pyperclip.copy(href_value)
    print("URL copied to clipboard")
  except Exception as e:
      print(f"An error occurred: {e}")
      raise e
  finally:
      # Close the browser
      driver.quit()

In [4]:
import yt_dlp
import shutil
import os

def download_music(url, base_path):
    # Define the destination file
    music_path = os.path.join(base_path, "music.mp3")

    if os.path.exists(music_path):
        return music_path

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'music.%(ext)s'
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

            # Move the song to the folder
            shutil.move("music.mp3", music_path)
    except Exception as e:
        print(f"An error occurred: {e}")
        raise e
    finally:
        return music_path

In [5]:
from audio_separator.separator import Separator

def separate_music(music_path, base_path):
  # Define result path
  instrumental_path = os.path.join(base_path, 'instrumental.wav')
  vocal_path = os.path.join(base_path, 'vocal.wav')

  if os.path.exists(instrumental_path) and os.path.exists(vocal_path):
    return instrumental_path, vocal_path

  # Initialize the Separator class (with optional configuration properties, below)
  separator = Separator()

  # Load a machine learning model (if unspecified, defaults to 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt')
  separator.load_model()

  # Perform the separation on specific audio files without reloading the model
  output_files = separator.separate(music_path)

  # Rename the output files
  os.rename(output_files[0], 'instrumental.wav')
  os.rename(output_files[1], 'vocal.wav')

  # Move the output files to the folderL
  shutil.move('instrumental.wav', instrumental_path)
  shutil.move('vocal.wav', vocal_path)

  return instrumental_path, vocal_path

In [6]:
import os
import requests

base_path = os.path.join(os.getcwd(), "drive", "MyDrive", "data_notebook", "music", "results")
base_url = "https://api.deezer.com/chart/0/tracks?index=0"
response = requests.get(base_url)

if response.status_code == 200:
  data = response.json()

  for i, track in enumerate(data["data"]):
        artist = track["artist"]["name"]
        title = track["title"]
        print(f"{i+1}. {artist} - {title}")
        
        base_path_song = create_folder(base_path, artist, title)
        song_url = get_url_music(title + " " + artist)

        music_path = download_music(song_url, base_path_song)
        instrumental_path, vocal_path = separate_music(music_path, base_path_song)
        
else:
    print("Error:", response.status_code)
    raise Exception("Failed to fetch the top tracks")

1. Raffaella Carrà - Pedro


WebDriverException: Message: Service /home/jovyan/.wdm/drivers/chromedriver/linux64/114.0.5735.90/chromedriver unexpectedly exited. Status code was: 127
